# **PosTech Inteligência Artificial Para Desenvolvedores - Fiap**
-  Projeto Challenge Fase 3 - Grupo 51
   

 # **Grupo 51/Alunos:**
 - RM355027 - José Hélio Araujo Andrade - helioandrade@hotmail.com
 - RM356210 - Bernardo Guimarães Tinti - betinti@hotmail.com

# Entregável

- [Repositório no GitHub]()
- [Vídeo YouTube]()

# Preparação do Ambiente

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets triton

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-wn8mnh9o/unsloth_604dba3963d84c80908747f661575ce0
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-wn8mnh9o/unsloth_604dba3963d84c80908747f661575ce0
  Resolved https://github.com/unslothai/unsloth.git to commit c3f4e9a87d964ecee1efd9963f497119edbefaab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
  

In [20]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import TextStreamer
import requests

amazonJsonDatasetPath = r'/content/drive/MyDrive/Pos_IA4Devs/Fine Tunning/Amazon_chunk_001.jsonl'
DATA_PATH = "/content/drive/MyDrive/Pos_IA4Devs/Fine Tunning/AmazonTitle_Processed.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Pos_IA4Devs/Fine Tunning/AmazonTitle_Output.json"
filePathProcessed = r'/content/drive/MyDrive/Pos_IA4Devs/Fine Tunning/AmazonTitle_Processed.json'
loraModelPath = "/content/drive/MyDrive/Pos_IA4Devs/Fine Tunning" # Caminho para armazenamento do modelo
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Pos_IA4Devs/Fine Tunning/formatted_news_dataset_chat_data.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


# Preparação do Dataset

## Funções

In [14]:
def extractDatasetToJsonStringDataset(filePath):
  data_processed = []

  with open(filePath, 'r', encoding='utf-8') as file:
    # Read the file content
    file_content = file.read()

    # Split the content into individual JSON objects
    json_objects = file_content.strip().split('\n')


    for line in json_objects:
      item = json.loads(line)
      story = item["title"]
      summary = item["content"]
      formatted_text = f"TALK ABOUT THIS PRODUCT.\n[|Title|] {story}[|eTitle|]\n\n[|content|]{summary}[|econtent|]"
      data_processed.append({"input": formatted_text})



  with open(DATA_PATH, 'w', encoding='utf-8') as fileProcessed:
      json.dump(data_processed, fileProcessed, ensure_ascii=False, indent=4)

In [16]:
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        news_start = full_text.find("[|Title|]") + len("[|Title|]")
        news_end = full_text.find("[|eTitle|]")
        summary_start = full_text.find("[|content|]") + len("[|content|]")
        summary_end = full_text.find("[|econtent|]")

        instruction = full_text.split('\n')[0]
        input_text = full_text[news_start:news_end].strip()
        response = full_text[summary_start:summary_end].strip()

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    for prompt in data:
        instruction, input_text, response = separate_text(prompt)
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

In [17]:
# Transformando o arquivo gerado em um array de strings com o campo de input
def convertJsonFileToArray(file_path):
  # Abrir e ler o conteúdo do arquivo JSON
  with open(file_path, 'r') as file:
      data = json.load(file)

  # Extrair os dados para um array
  extracted_data = []
  for entry in data:
      extracted_data.append(entry['input'])

  print(extracted_data[:5])
  return extracted_data

## Execução do Tratamento de Dados

In [18]:
# Extração dos dados do Dataset Amazon
extractDatasetToJsonStringDataset(amazonJsonDatasetPath)

# Preparação dos dados para treinaemtno da LLM
format_dataset_into_model_input(convertJsonFileToArray(DATA_PATH))

['TALK ABOUT THIS PRODUCT.\n[|Title|] Girls Ballet Tutu Neon Pink[|eTitle|]\n\n[|content|]High quality 3 layer ballet tutu. 12 inches in length[|econtent|]', "TALK ABOUT THIS PRODUCT.\n[|Title|] Mog's Kittens[|eTitle|]\n\n[|content|]Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.[|econtent|]", 'TALK ABOUT THIS PRODUCT.\n[|Title|] Girls Ballet Tutu Neon Blue[|eTitle|]\n\n[|content|]Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes or just for fun![|econtent|]', 'TALK ABOUT THIS PRODUCT.\n[|Title|] The Prophet[|eTitle|]\n\n[|content|]In a distant, timeless place, a mysterious pr

# Chamada do Foundation Model

## Carrega um modelo pré-treinado junto com seu tokenizador

Este código carrega um modelo de linguagem pré-treinado, utilizando a biblioteca `FastLanguageModel` para configurar tanto o modelo quanto o tokenizer. O modelo que está sendo carregado é uma variante compactada para otimização de memória e desempenho, sendo executado em `4 bits` para economizar memória e acelerar o processamento.

Modelo escolhido foi o `llama-3-8b-bnb-4bit`. <br>
O modelo é baseado na arquitetura LLaMA (Large Language Model Meta AI), possivelmente com 8 bilhões de parâmetros (representado por "3-8b") e otimizado para ser carregado e executado em um formato de 4 bits, o que significa que ele foi quantizado para reduzir o uso de memória, tornando-o mais leve. <br>
O termo bnb refere-se ao uso da técnica BitsAndBytes, uma ferramenta para compressão e execução de modelos de forma eficiente, especialmente em dispositivos de menor capacidade.

In [9]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Nome do modelo a ser carregado.
    max_seq_length = max_seq_length,  # Define o comprimento máximo da sequência de tokens que o modelo pode processar em uma única passagem.
    dtype = dtype,                    # O parâmetro dtype especifica o tipo de dado (data type) que será usado para representar os pesos do modelo.
    load_in_4bit = load_in_4bit,      # Este parâmetro ativa o carregamento do modelo em um formato compactado de 4 bits.
)

Dataset salvo em /content/drive/MyDrive/Pos_IA4Devs/Fine Tunning/AmazonTitle_Output.json
==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Utilizando uma técnica chamada Parameter Efficient Fine-Tuning (PEFT). <br>
A biblioteca PEFT é projetada para melhorar a eficiência do ajuste fino de grandes modelos de linguagem, reduzindo o número de parâmetros treináveis, o que, por sua vez, torna o treinamento mais rápido e econômico em termos de memória.

In [19]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,           #especifica a dimensão da matriz de rank baixo usada durante o ajuste fino.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,  # hiperparâmetro usado para escalar a adaptação LoRA. Ele controla a magnitude das atualizações feitas nos parâmetros LoRA
    lora_dropout = 0, # Indica que não será aplicado dropout nas camadas ajustadas pela LoRA
    bias = "none",    # Este parâmetro controla se os viéses das camadas devem ser treinados ou não. No caso não serão.

    use_gradient_checkpointing = "unsloth", # Técnica que economiza memória durante o treinamento,
    random_state = 3407,                    # Define um estado aleatório para garantir reprodutibilidade.
    use_rslora = False,
    loftq_config = None,                    # Este parâmetro refere-se a uma configuração para uma técnica chamada (Low-Rank Adaptation for Quantization)
)

Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Carregando modelo sem o processo de Fine-Tuning

<br>
Carregando um modelo com os mesmo parâmetros, que não será feito o processo de Fine-Tuning. <br>
Ele será usado a nível de comparação entre um modelo com e sem treinamento.

In [ ]:
#Criando um model sem realizar o fine tune

modelClean, tokenizerClean = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

modelClean = FastLanguageModel.get_peft_model(
    modelClean,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# Execução do Fine-Tuning

In [22]:
# Prompt a ser usado nas consultas ao modelo
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Formatação do dataset para a tokenização e em sequência treinamento do modelo

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts        =         []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

FileNotFoundError: Unable to find '/content/drive/MyDrive/Pos_IA4Devs/Fine Tunning/formatted_news_dataset_chat_data.json'

In [21]:
# Criando o treinamento

trainer = SFTTrainer(
    model = model,                    # O modelo de linguagem que será ajustado.
    tokenizer = tokenizer,            # O tokenizador usado para converter os dados de texto em tokens que podem ser entendidos pelo modelo.
    train_dataset = dataset,          # O conjunto de dados de treinamento que será usado para ajustar o modelo.
    dataset_text_field = "text",      # Este parâmetro indica qual campo no conjunto de dados contém o texto que será utilizado para o treinamento.
    max_seq_length = max_seq_length,  # O comprimento máximo da sequência de tokens que o modelo pode processar por vez.
    dataset_num_proc = 2,             # Este parâmetro define o número de processos paralelos a serem usados para o pré-processamento do conjunto de dados.
    packing = False,                  # Este parâmetro indica se as sequências de entrada no conjunto de dados devem ser agrupadas ou não. No caso, cada entrada será processada individualmente.
    args = TrainingArguments(   # Configurando hiperparâmetros importantes que controlam como o treinamento será realizado.
        per_device_train_batch_size = 2,    # Tamanho do lote de treinamento para cada dispositivo (por exemplo, GPU ou CPU).
        gradient_accumulation_steps = 4,    # Número de passos de acumulação de gradiente antes de realizar uma atualização nos parâmetros do modelo.
        warmup_steps = 5,                   # Número de passos de aquecimento (warm-up) para o agendador de taxa de aprendizado.
        max_steps = 60,                     # Número máximo de passos de treinamento.
        learning_rate = 2e-4,               # A taxa de aprendizado define o quão rapidamente o modelo ajusta seus parâmetros em resposta ao erro (perda) durante o treinamento. (no caso 0,0002).
        fp16 = not is_bfloat16_supported(), # Se o dispositivo não suportar o formato bfloat16, será utilizado o fp16. Isso reduz o uso de memória e aumenta a eficiência computacional.
        bf16 = is_bfloat16_supported(),     # Verifica se o dispositivo suporta bfloat16
        logging_steps = 1,                  # Número de passos entre cada registro de log durante o treinamento.
        optim = "adamw_8bit",               # Define o otimizador a ser utilizado durante o treinamento.
        weight_decay = 0.01,                # Taxa de decaimento de peso aplicada para evitar overfitting.
        lr_scheduler_type = "linear",       # Define o tipo de agendador de taxa de aprendizado.
        seed = 3407,                        # Semente aleatória usada para garantir a reprodutibilidade dos experimentos.
        output_dir = "outputs",             # Diretório onde os resultados do treinamento, como os pesos do modelo ajustado e os logs, serão salvos.
    ),
)

NameError: name 'dataset' is not defined

In [42]:
# Executando o treinamento

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 75,062 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.970600
2,3.431400
3,2.618000
4,2.290100
5,2.470300
6,1.810800
7,2.486300
8,1.990200
9,2.123900
10,1.711200


In [43]:
# Local de armazagenagem do modelo após o treinamento

model.save_pretrained(loraModelPath)
tokenizer.save_pretrained(loraModelPath)

('/content/drive/MyDrive/Pos_IA4Devs/Fine Tunning/tokenizer_config.json',
 '/content/drive/MyDrive/Pos_IA4Devs/Fine Tunning/special_tokens_map.json',
 '/content/drive/MyDrive/Pos_IA4Devs/Fine Tunning/tokenizer.json')

## Usando o modelo salvo já treinado

In [5]:
if True:
    from unsloth import FastLanguageModel
    # Recarregando o modelo já treinado anteriormente
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = loraModelPath, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Geração de Respostas

## Definição do Prompt

In [24]:
command = "Using your knowledge base talk about this prodcut. If you can’t find relevant information to answer the question, respond with, “Sorry, I don’t have any information on this question.”"

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        command,
        contentInput,
        ""
    )
], return_tensors = "pt").to("cuda")

## Gerando respostas ao usuário

In [ ]:
contentInput = input("Enter your question ...")

print()
print("----- Unsloth com Fine Tune --------------------------------------------------------------------")
print()

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

print()
print("----- Unsloth Sem Fine Tune ------------------------------------------------------------------------------")
print()

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizerClean)
_ = modelClean.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)